In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.1 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy
from operator import itemgetter

In [ ]:
classifier = pipeline(task = "fill-mask", model = 'xlm-roberta-base', device = 'cuda:0')
# classifier = pipeline(task = "fill-mask", model = 'bert-base-uncased', device = 'cuda:0')

In [ ]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.tokenizer

Загрузка датасетов:


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving good_verb_pattern.txt to good_verb_pattern.txt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving good_lex_item_choice.txt to good_lex_item_choice.txt


Функция маскирования:

In [ ]:
def masking(row):
    all_scores = []
    for i in range(len(row)): # каждое немаск предложение
        score = []
        sent = list(tokenizer(row[i]))
        len_sent = len(sent)
        for j in range(len_sent): # каждое маск предложение
    #         sent_id.append(i)
            sent_list = [str(word) for word in sent]
            masked = sent_list[j]
            sent_list[j] = "<mask>"
            sent_list = " ".join(word for word in sent_list)
            for pred_token in classifier(sent_list):
                if pred_token['token_str'] == masked:
                    score.append(pred_token['score'])
        all_scores.append(sum(score) / len_sent)
    return all_scores



In [1]:
df = pd.read_csv("verb_pattern.tsv", delimiter = "\t")

all_score_good_verb_pattern = masking(df["sentence_good"])
all_score_bad_verb_pattern = masking(df["sentence_bad"])

result_verb_pattern = []
for i in range(len(all_score_good_verb_pattern)):
    if all_score_good_verb_pattern[i] > all_score_bad_verb_pattern[i]:
        result_verb_pattern.append(1)
    else:
        result_verb_pattern.append(0)

NameError: ignored

In [3]:
accuracy_verb_pattern = sum([int(i) for i in result_verb_pattern_errors])/len(result_verb_pattern_errors)

NameError: ignored

In [ ]:
print("Accuracy: ", accuracy_verb_pattern)

Accuracy:  0.7973856209150327


In [ ]:
df = pd.read_csv("lex_item_choice.tsv", delimiter = "\t")

all_score_good_lex_item_choice = masking(df["sentence_good"])
all_score_bad_lex_item_choice = masking(df["sentence_bad"])

result_lex_item_choice = []
for i in range(len(all_score_good_lex_item_choice)):
    if float(all_score_good_lex_item_choice[i]) > all_score_bad_lex_item_choice[i]:
        result_lex_item_choice.append(1)
    else:
        result_lex_item_choice.append(0)

In [ ]:
accuracy_lex_item_choice = sum([int(i) for i in result_lex_item_choice])/len(result_lex_item_choice)

In [ ]:
print("Accuracy: ", accuracy_lex_item_choice)

Accuracy:  0.7190838389881207


In [ ]:
result_all = result_lex_item_choice + result_verb_pattern_errors
accuracy_all = sum([int(i) for i in result_all])/len(result_all)

data = {"field": ["verb_pattern", "lex_item_choice", "semantics"], "accuracy": [accuracy_verb_pattern, accuracy_lex_item_choice, accuracy_all]}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("sem_results_roberta.csv", index=False) # creating a new file